In [2]:
import snowflake.connector

In [3]:
#conn is the object that connects you to your Snowflake account
conn = snowflake.connector.connect(
user='eoriley169',
password='2YzrtnhWKL6r5dT',
account='afaypkh-rjb48354'
)

In [4]:
#cs is a database cursor object for execute and fetch operations
cs = conn.cursor()

In [7]:
## Create Snowflake Objects
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

In [8]:
#Create a database
cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

In [9]:
#Create a schema
cs.execute("CREATE SCHEMA IF NOT EXISTS testschema")

In [10]:
#Next create a table and insert data (this is regular DDL and DML):
cs.execute(
"CREATE OR REPLACE TABLE "
"test_table(col1 integer, col2 string)")
cs.execute(
"INSERT INTO test_table(col1, col2) "
"VALUES (123, 'test string1'), (456, 'test string2')")

In [12]:
#run a SELECT query and view the result
cs.execute('SELECT * FROM test_table')
print(cs.fetchmany(2))

[(123, 'test string1'), (456, 'test string2')]


In [5]:
# --- Add this to ETL_Snowflake.py (after your existing connection code) ---

import snowflake.connector

def use_first_existing_db(cs, candidate_names):
    """
    Try each candidate database name; return the first one that exists.
    Falls back to SHOW DATABASES LIKE 'WEATHER%ENVIRONMENT%' if none match exactly.
    """
    # Try exact names first
    for db in candidate_names:
        cs.execute(f"SHOW DATABASES LIKE '{db}'")
        if cs.fetchone():
            return db

    # Broad match as a fallback (handles custom naming)
    cs.execute("SHOW DATABASES LIKE 'WEATHER%ENVIRONMENT%'")
    row = cs.fetchone()
    if row:
        # SHOW DATABASES returns tuples; the 2nd field is the database NAME
        # Typical order: created_on, name, is_default, ...
        return row[1]

    raise RuntimeError(
        "Could not find a WEATHER...ENVIRONMENT database. "
        "Verify the Snowflake Marketplace subscription/name in the web UI."
    )

def print_table_sample(cs, fully_qualified_table, sample_rows=5):
    """
    Print column names, a small sample, and total row count for the given table.
    """
    print(f"\n=== {fully_qualified_table} ===")

    # Sample rows
    cs.execute(f"SELECT * FROM {fully_qualified_table} LIMIT {sample_rows}")
    rows = cs.fetchall()
    col_names = [desc[0] for desc in cs.description]
    print("Columns:", ", ".join(col_names))
    for i, r in enumerate(rows, 1):
        print(f"{i:>2}: {r}")

    # Row count (fast metadata-style count is not available; run COUNT(*))
    cs.execute(f"SELECT COUNT(*) FROM {fully_qualified_table}")
    total = cs.fetchone()[0]
    print(f"Total rows: {total}")

def print_cybersyn_weather_tables(conn, warehouse="MY_FIRST_WAREHOUSE"):
    """
    Uses only the Snowflake Python connector to:
    - set the warehouse,
    - locate the Marketplace database,
    - switch to CYBERSYN schema,
    - and print samples + counts from the two NOAA tables.
    """
    with conn.cursor() as cs:
        # Ensure a warehouse is set (you created MY_FIRST_WAREHOUSE earlier in your file)
        cs.execute(f"USE WAREHOUSE {warehouse}")

        # Part 2, step 7 database names: assignment shows WEATHER__ENVIRONMENT (double underscore),
        # but some accounts show WEATHER_ENVIRONMENT (single underscore).
        candidate_dbs = ["WEATHER__ENVIRONMENT", "WEATHER_ENVIRONMENT"]
        db_name = use_first_existing_db(cs, candidate_dbs)
        cs.execute(f"USE DATABASE {db_name}")

        # Schema from the brief is CYBERSYN
        cs.execute("USE SCHEMA CYBERSYN")

        # Fully qualified table names (no quotes needed since identifiers are simple/upper)
        tables = [
            "NOAA_WEATHER_METRICS_TIMESERIES",
            "NOAA_WEATHER_STATION_INDEX",
        ]
        for t in tables:
            fqtn = f"{db_name}.CYBERSYN.{t}"
            print_table_sample(cs, fqtn, sample_rows=5)

# --- Run it ---
if __name__ == "__main__":
    # Reuse your existing `conn` from above in ETL_Snowflake.py
    # Example assumes you already did:
    # conn = snowflake.connector.connect(user=..., password=..., account=...)
    try:
        print_cybersyn_weather_tables(conn, warehouse="MY_FIRST_WAREHOUSE")
    finally:
        try:
            conn.close()
        except Exception:
            pass



=== WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES ===
Columns: NOAA_WEATHER_STATION_ID, VARIABLE, VARIABLE_NAME, DATE, DATETIME, VALUE, UNIT
 1: ('USR0000CBAI', 'maximum_temperature_represents_highest_or_lowest_hourly_temperature_or_the_average_of_hourly_values', 'Maximum Temperature (Represents Highest or Lowest Hourly Temperature or the Average of Hourly Values)', datetime.date(2005, 7, 3), datetime.datetime(2005, 7, 3, 0, 0), Decimal('25.000000'), 'Degrees Celsius')
 2: ('SWM00002584', 'precipitation', 'Precipitation', datetime.date(2003, 11, 23), datetime.datetime(2003, 11, 23, 0, 0), Decimal('13.100000'), 'Millimeters')
 3: ('RMW00040604', 'weather_type_rain_may_include_freezing_rain_drizzle_and_freezing_drizzle', 'Weather Type: Rain (may include freezing rain, drizzle, and freezing drizzle)', datetime.date(2005, 8, 25), datetime.datetime(2005, 8, 25, 0, 0), Decimal('1.000000'), 'Boolean')
 4: ('USW00026501', 'average_wind_speed', 'Average Wind Speed', datetime.da